# Klasy

# Metody __new__ & __init__

In [55]:
class Dummy:
    def __new__(cls, *args):
        print(f"Dummy.__new__({cls}, {args}) has been called...")
        obj = super().__new__(cls) # calling __new__ from object
        obj.extra_attribute = "Extra Attribute"
        print(f"Object {obj} has been created...")
        return obj
    
    def __init__(self, *args):
        print(f"Object's __dict__: {self.__dict__}")
        print(f"Dummy.__init__({self}, {args})...")
        self.args = args
        print(f"Object's __dict__: {self.__dict__}")

In [56]:
d = Dummy(1, "One")

Dummy.__new__(<class '__main__.Dummy'>, (1, 'One')) has been called...
Object <__main__.Dummy object at 0x0000024D3872B8D0> has been created...
Object's __dict__: {'extra_attribute': 'Extra Attribute'}
Dummy.__init__(<__main__.Dummy object at 0x0000024D3872B8D0>, (1, 'One'))...
Object's __dict__: {'extra_attribute': 'Extra Attribute', 'args': (1, 'One')}


## Kiedy używać `__new__`?

In [57]:
class UppercaseTuple(tuple):
    def __init__(self, list) -> None:
        print(f"Start changes for {list}")

        for i, item in enumerate(list):
            self[i] = item.upper()

In [58]:
UppercaseTuple(["hello", "world", "!"])

Start changes for ['hello', 'world', '!']


TypeError: 'UppercaseTuple' object does not support item assignment

In [59]:
class UppercaseTuple(tuple):
    def __new__(cls, list):
        print(f"Start changes for {list}")
        new_content = [item.upper() for item in list]       
        return super().__new__(cls, new_content)

In [60]:
UppercaseTuple(["hello", "world", "!"])

Start changes for ['hello', 'world', '!']


('HELLO', 'WORLD', '!')

# Dostęp do atrybutów

In [73]:
from typing import Any


class Record:
    def __init__(self):
        # Nie możemy użyć poniższego kodu:
        #     self._d = {}
        # ponieważ zakończyłby się on rekurencyjnym wywoływaniem metody __setattr__
        super().__setattr__('_dict', {})

    def __getattribute__(self, name: str) -> Any:
        print('getting by __getattribute__', name)
        return super().__getattribute__(name)

    def __getattr__(self, name):
        print('getting', name)
        return self._dict[name]
    
    def __setattr__(self, name, value):
        print('setting', name, 'to', value)
        self._dict[name] = value
        
    def __delattr__(self, name):
        print('deleting', name)
        del self._dict[name]

In [74]:
person = Record()

In [75]:
person.first_name = "John"

setting first_name to John
getting by __getattribute__ _dict


In [76]:
person._dict

getting by __getattribute__ _dict


{'first_name': 'John'}

In [77]:
person.first_name

getting by __getattribute__ first_name
getting first_name
getting by __getattribute__ _dict


'John'

In [65]:
del person.first_name

deleting first_name


In [67]:
person._dict

{}

In [72]:
person.__dict__

{'_dict': {'first_name': 'John'}}

In [84]:
import datetime


class Foo:
    def __init__(self):
        self.a = "Foo.a"

    def __getattr__(self, attribute):
        return f"You asked for {attribute}, but I'm giving you a default str"


class Bar:
    attribute_access_log = []

    def __init__(self):
        self.a = "Bar.a"

    def __getattribute__(self, attribute):
        Bar.attribute_access_log.append(
            f"Access to {self}.{attribute} at {datetime.datetime.now()}")
        print(f"You asked for {attribute}")
        return super().__getattribute__(attribute)

In [85]:
foo = Foo()

In [86]:
foo.a

'Foo.a'

In [87]:
foo.b

"You asked for b, but I'm giving you a default str"

In [88]:
bar = Bar()

You asked for __class__
You asked for __class__
You asked for __class__
You asked for __class__


In [89]:
bar.a

You asked for a


'Bar.a'

In [90]:
bar.b

You asked for b


AttributeError: 'Bar' object has no attribute 'b'

In [91]:
bar.attribute_access_log

You asked for attribute_access_log


['Access to <__main__.Bar object at 0x0000024D387AE010>.__class__ at 2023-05-10 13:35:49.870746',
 'Access to <__main__.Bar object at 0x0000024D387AE010>.__class__ at 2023-05-10 13:35:49.870746',
 'Access to <__main__.Bar object at 0x0000024D387AE010>.__class__ at 2023-05-10 13:35:49.870746',
 'Access to <__main__.Bar object at 0x0000024D387AE010>.__class__ at 2023-05-10 13:35:49.870746',
 'Access to <__main__.Bar object at 0x0000024D38214D90>.__dict__ at 2023-05-10 13:36:04.942671',
 'Access to <__main__.Bar object at 0x0000024D38214D90>.__class__ at 2023-05-10 13:36:04.942671',
 'Access to <__main__.Bar object at 0x0000024D38214D90>.__class__ at 2023-05-10 13:36:04.942671',
 'Access to <__main__.Bar object at 0x0000024D38214D90>.__class__ at 2023-05-10 13:36:04.942671',
 'Access to <__main__.Bar object at 0x0000024D38214D90>.__class__ at 2023-05-10 13:36:04.942671',
 'Access to <__main__.Bar object at 0x0000024D38214D90>.a at 2023-05-10 13:36:06.167045',
 'Access to <__main__.Bar obj

# Składowe private i protected

In [108]:
class BankAccount:
    def __init__(self, initial_balance: float):
        self.__balance = initial_balance
        self._withdrawal_limit = 100

    @property
    def balance(self) -> float:
        return self.__balance
    
    def __check_amount(self, amount: float):  # private only by convention
        if amount <= 0:
            raise ValueError("value cannot be negative")
    
class DebitBankAccount(BankAccount):
    def __init__(self, initial_balance: float):
        super().__init__(initial_balance)

    def withdraw(self, amount: float):
        self.__check_amount(amount)
        if amount > self._withdrawal_limit:
            raise ValueError("amount too large")

In [100]:
account_1 = BankAccount(1000)

In [101]:
account_1.balance

1000

In [102]:
account_1.balance = 100

AttributeError: property 'balance' of 'BankAccount' object has no setter

In [96]:
account_1._BankAccount__balance = 100

In [97]:
account_1.balance

100

In [106]:
debit_account = DebitBankAccount(1000)
debit_account.withdraw(400)

ValueError: amount too large

In [109]:
debit_account.withdraw(-29)

ValueError: value cannot be negative

# Składowe statyczne

In [114]:
class CountedObject(object):
    count = 0   # statyczna składowa
    
    def __init__(self):
        CountedObject.count += 1

    def __del__(self):
        CountedObject.count -= 1
    
    @staticmethod  # statyczna metoda
    def get_count():
        return CountedObject.count

In [116]:
lst = [CountedObject() for i in range(10)]

In [117]:
del lst[0]

In [118]:
CountedObject.count

9

In [139]:
class Person:
    age = 18 # class member as default value

    def __init__(self, name: str):
        self.name = name    
        

In [140]:
p1 = Person("John")

In [141]:
p1.name

'John'

In [142]:
p1.age # getting default value

18

In [135]:
p1.age = 66 # assigning new value for the instance

In [136]:
p1.age

66

In [137]:
p2 = Person("Eve")

In [138]:
p2.age # default value

18

In [152]:
class Date:
    year = 2023

    def __init__(self, day, month, year = None):
        self.day = day
        self.month = month
        if year:
            self.year = year 
    
    @classmethod
    def from_string(cls, date_as_string):        
        day, month, year = date_as_string.split('-')
        return cls(int(day), int(month), int(year)) # utworzenie instancji klasy cls

    @classmethod
    def update_default_year(cls, value):
        cls.year = value

In [155]:
d1 = Date(20, 1)

In [156]:
d1.year

2023

In [157]:
d2 = Date(20, 3)

In [158]:
d2.year

2023

In [159]:
Date.update_default_year(2222)

In [161]:
d1.__dict__

{'day': 20, 'month': 1}

In [162]:
Date.__dict__

mappingproxy({'__module__': '__main__',
              'year': 2222,
              '__init__': <function __main__.Date.__init__(self, day, month, year=None)>,
              'from_string': <classmethod(<function Date.from_string at 0x0000024D388F6980>)>,
              'update_default_year': <classmethod(<function Date.update_default_year at 0x0000024D388F68E0>)>,
              '__dict__': <attribute '__dict__' of 'Date' objects>,
              '__weakref__': <attribute '__weakref__' of 'Date' objects>,
              '__doc__': None})

In [160]:
d1.year

2222

# Deskryptor

## Non-data descriptor

In [163]:
import os

class DirectorySize:
    def __get__(self, instance, owner_class):
        return len(os.listdir(instance.directory_name))
    

class Directory: # owner class
    size = DirectorySize() # descriptor instance

    def __init__(self, directory_name):
        self.directory_name = directory_name # regular instance attribute

In [164]:
local_dir = Directory('.')

In [165]:
local_dir.__dict__

{'directory_name': '.'}

In [166]:
local_dir.size

3

## Data descriptor

In [184]:
import logging

logging.basicConfig(level=logging.INFO)

class LoggedAccess: # Data descriptor

    def __set_name__(self, owner, name):
        self.public_name = name
        self.private_name = '_' + name
        logging.info('Setting names: %r and %r', self.public_name, self.private_name)

    def __get__(self, instance, owner_class=None):
        value = getattr(instance, self.private_name)
        logging.info('Accessing %r.%r giving %r', instance, self.public_name, value)
        return value

    def __set__(self, instance, value):
        logging.info('Updating %r.%r to %r', instance, self.public_name, value)
        setattr(instance, self.private_name, value)

In [185]:
class Person:
    name = LoggedAccess()
    age = LoggedAccess()
    
    # Descriptor instance
    def __init__(self, name, age):
        self.name = name                # Regular instance attribute
        self.age = age                  # Calls __set__()

    def birthday(self):
        self.age += 1  

INFO:root:Setting names: 'name' and '_name'
INFO:root:Setting names: 'age' and '_age'


In [179]:
p1 = Person("John", 42)

INFO:root:Updating <__main__.Person object at 0x0000024D388B6910>.'__name' to 'John'
INFO:root:Updating <__main__.Person object at 0x0000024D388B6910>.'__age' to 42


In [180]:
p1.__dict__

{'__name': 'John', '__age': 42}

In [181]:
p1.age

INFO:root:Accessing <__main__.Person object at 0x0000024D388B6910>.'__age' giving 42


42

In [182]:
p1.birthday()

INFO:root:Accessing <__main__.Person object at 0x0000024D388B6910>.'__age' giving 42
INFO:root:Updating <__main__.Person object at 0x0000024D388B6910>.'__age' to 43


In [173]:
p1.age

INFO:root:Accessing <__main__.Person object at 0x0000024D3884A4D0>.'age' giving 43


43

In [189]:
from typing import Any, Callable


class ValidatedAttribute:
    """Data descriptor"""

    def __init__(self, validator: Callable[..., bool]) -> None:
        self.validator = validator

    def __set_name__(self, owner, name):
        self.private_name = '_' + name
        print(f'Setting a private name: {self.private_name}')

    def __get__(self, instance: Any, owner: type):
        print("ValidatedAttribute.__get__")
        print(f"..........self: {self}")
        print(f"......instance: {instance}")
        print(f".........owner: {owner}")
        #return getattr(instance, self.private_name)
        return "42"

    def __set__(self, instance, value):
        print("ValidatedAttribute.__set__")
        print(f"..........self: {self}")
        print(f"......instance: {instance}")
        print(f".........value: {value}")
        if self.validator(value):
            setattr(instance, self.private_name, value)
        else:
            raise AttributeError("Attribute validation failed")
        
    def __delete__(self, instance: Any) -> None:
        print("ValidatedAttribute.__delete__")
        print(f"..........self: {self}")
        print(f"......instance: {instance}")
        setattr(instance, self.private_name, None)


# owner class
class Data:
    even = ValidatedAttribute(lambda n: n % 2 == 0) # must be even

    def __init__(self, value):
        self.even = value

Setting a private name: _even


In [187]:
d1 = Data(24)

ValidatedAttribute.__set__
..........self: <__main__.ValidatedAttribute object at 0x0000024D36C8A210>
......instance: <__main__.Data object at 0x0000024D387D4B90>
.........value: 24


In [188]:
d1.even = 13

ValidatedAttribute.__set__
..........self: <__main__.ValidatedAttribute object at 0x0000024D36C8A210>
......instance: <__main__.Data object at 0x0000024D387D4B90>
.........value: 13


AttributeError: Attribute validation failed

# Slots

In [ ]:
class Pixel:
    # __slots__ =  ('x', 'y')

    def __init__(self, x, y):
        self.x = x
        self.y = y


class ColorPixel(Pixel):
    __slots__ = ('color')

    def __init__(self, x, y, color = 0):
        super().__init__(x, y)
        self.color = color

In [ ]:
p1 = Pixel(100, 200)

In [ ]:
p1.x

In [ ]:
p1.__dict__

In [ ]:
p1.z = 100

In [ ]:
cp = ColorPixel(100, 300)

In [ ]:
cp.__dict__

In [ ]:
cp.__slots__

In [ ]:
cp.x

In [ ]:
ColorPixel.__slots__

# Dziedziczenie

In [ ]:
class A:
    def __init__(self):
        print("A")

class B(A):
    def __init__(self):
        super().__init__()
        print("B")

class C(A):
    def __init__(self):
        super().__init__()
        print("C")

class D(B, C):
    def __init__(self):
        super().__init__()
        print("D")

In [ ]:
D.__mro__

## Mixins

In [ ]:
class ComparableMixin:
    def __ne__(self, other):
        return not (self == other)
    
    def __le__(self, other):
        return self < other or (self == other)
    
    def __gt__(self, other):
        return not self <= other
    
    def __ge__(self, other):
        return self > other or self == other
    

class MyInteger(ComparableMixin):  # klasą bazową jest "object"
    def __init__(self, i):
        self.i = i
    
    def __lt__(self, other):
        return self.i < other.i
    
    def __eq__(self, other):
        return self.i == other.i

In [ ]:
m1 = MyInteger(42)
m2 = MyInteger(665)

In [ ]:
m1 == m2

In [ ]:
m1 != m2

In [ ]:
m1 > m2

In [ ]:
import functools

@functools.total_ordering
class MyFloat:  
    def __init__(self, i: float):
        self.i = i
    
    def __lt__(self, other):
        return self.i < other.i
    
    def __eq__(self, other):
        return self.i == other.i

In [ ]:
mf1 = MyFloat(1.33)
mf2 = MyFloat(3.14)

In [ ]:
mf1 > mf2

# Dziedziczenie

In [59]:
class A:
    def __init__(self):
        print("A")

class B(A):
    def __init__(self):
        super().__init__()
        print("B")

class C(A):
    def __init__(self):
        super().__init__()
        print("C")

class D(B, C):
    def __init__(self):
        super().__init__()
        print("D")

In [60]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

## Mixins

In [61]:
class ComparableMixin:
    def __ne__(self, other):
        return not (self == other)
    
    def __le__(self, other):
        return self < other or (self == other)
    
    def __gt__(self, other):
        return not self <= other
    
    def __ge__(self, other):
        return self > other or self == other
    

class MyInteger(ComparableMixin):  # klasą bazową jest "object"
    def __init__(self, i):
        self.i = i
    
    def __lt__(self, other):
        return self.i < other.i
    
    def __eq__(self, other):
        return self.i == other.i

In [63]:
m1 = MyInteger(42)
m2 = MyInteger(665)

In [64]:
m1 == m2

False

In [65]:
m1 != m2

True

In [66]:
m1 > m2

False

In [67]:
import functools

@functools.total_ordering
class MyFloat:  
    def __init__(self, i: float):
        self.i = i
    
    def __lt__(self, other):
        return self.i < other.i
    
    def __eq__(self, other):
        return self.i == other.i

In [68]:
mf1 = MyFloat(1.33)
mf2 = MyFloat(3.14)

In [69]:
mf1 > mf2

False

## Abstract Base Classes

In [106]:
import abc
from collections import namedtuple
Coord = namedtuple('Coord', 'x y')

class Shape(abc.ABC):

    @abc.abstractmethod
    def move(self, dx: int, dy: int) -> None:
        raise NotImplementedError()

    @abc.abstractmethod
    def draw(self) -> None:
        raise NotImplementedError()
    
    @abc.abstractmethod
    def area(self):
        pass
        


class ShapeBase(Shape):
    def __init__(self, coord: Coord) -> None:
        self.coord = coord

    def move(self, dx: int, dy: int):
        self.coord = Coord(self.coord.x + dx, self.coord.y + dy)


class Rectangle(ShapeBase):
    def __init__(self, coord: Coord, w: int, h: int):
        super().__init__(coord)
        self.width = w
        self.height = h

    def draw(self):        
        print(f"Drawing Rectangle at {self.coord} with width {self.width} and height {self.height}")

    def area(self):
        return self.width * self.height


class Circle(ShapeBase):
    def __init__(self, coord: Coord, r: int):
        super().__init__(coord)
        self.radius = r        

    def draw(self):
        print(f"Drawing Circle at {self.coord} with width {self.radius}")

    def area(self):
        return self.radius ** 2 * 3.14

class Square(Shape):
    def __init__(self, coord: Coord, size: int):
        self.__rect = Rectangle(coord, size, size)

    def move(self, dx, dy):
        self.__rect.move(dx, dy)

        def draw(self):
        self.__rect.draw()

# class Square(Rectangle):
#     def __init__(self, coord: Coord, size: int):
#         super().__init__(coord, size, size)

#     @property
#     def height(self):
#         return self.width

#     @height.setter
#     def height(self, value):         
#          self.width = value

#     @property
#     def width(self):
#         return self.height

#     @width.setter
#     def width(self, value):         
#          self.height = value


In [107]:
rect = Rectangle(Coord(100, 200), 10, 20)

rect.move(150, 200)
rect.draw()

Drawing Rectangle at Coord(x=250, y=400) with width 10 and height 20


In [108]:
c = Circle(Coord(500, 200), 100)

In [109]:
from typing import List


def render(shapes: List[Shape]):
    for shp in shapes:
        shp.draw()

In [110]:
shapes = [rect, c]

In [111]:
render(shapes)

Drawing Rectangle at Coord(x=250, y=400) with width 10 and height 20
Drawing Circle at Coord(x=500, y=200) with width 100


In [100]:
sb = ShapeBase()

TypeError: Can't instantiate abstract class ShapeBase with abstract method draw

In [112]:
def test_evil(rect: Rectangle):
    rect.width = 10
    rect.height = 20

    assert rect.area() == 200    

In [113]:
test_evil(rect)

In [114]:
square = Square(Coord(0, 0), 100)


: 

: 

In [105]:
test_evil(square)

In [ ]:
sq = Square(Coord(10, 20), 100)

In [ ]:
sq.width = 200
sq.height = 400

In [ ]:
def client(r: Rectangle):
    r.width = 100
    r.height = 300

    if isinstance(r, Square):
        pass

    if (r.area() < 400):
        pass